In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import os
import pandas as pd
import numpy as np
from pathlib import Path

labels = ['allrock', 'blues', 'country', 'electronic', 'hip-hop', \
          'jazz', 'latin-american', 'pop', 'r-n-b', 'reggae', 'shanson', 'world']

labels_small = ['allrock_small', 'blues_small', 'country_small', 'electronic_small', 'hip-hop_small', \
          'jazz_small', 'latin-american_small', 'pop_small', 'r-n-b_small', 'reggae_small', 'shanson_small', 'world_small']

x_lim = 431  #(22050/512) * 10 = 431 отсчет на 10 секунд звучания
y_lim = 15   #n_mfcc

n_songs = 30 #песен у каждого исполнителя
n_artists = 300 #исполнителей в каждом жанре. Но может быть чуть меньше.

In [ ]:
#записать все данные в один файл

filepath_out = os.path.join(os.getcwd, '431na15_full.csv')
song = pd.DataFrame()
#root = os.getcwd()
root = os.getcwd #'/gdrive/My Drive/DATASETS/genres/genres_all'
print('Загрузка:')
for root_dir, dirs, files in os.walk(root):
    for file in files:
        if Path(file).suffix == '.csv' and Path(file).stem in labels: #labels_small:
            filepath = os.path.join(root_dir, file)
            print(file)
            sound = pd.read_csv(filepath) #читаем файл с mfcc конкретного жанра
            #sound = sound.iloc[:int(n_songs * n_artists * coeff)] #оставляем только koeff песен так как все не влезает в ОЗУ
            #sound = sound.sample(frac=1).reset_index(drop=True) #перемешиваем песни
            sound['class'] = Path(file).stem
            song = pd.concat([song, sound], ignore_index = True)
            song.to_csv(filepath_out, index=False, mode = 'a')
            
#12930*15 = 1 исполнитель (30 песен)
#431*15 = 1 песня (10 секунд)

Загрузка:
world.csv
allrock.csv


KeyboardInterrupt: ignored

In [ ]:
song = pd.read_csv( os.path.join(os.getcwd, '431na15_full.csv'), index_col = None) #'/gdrive/My Drive/DATASETS/genres/genres_all/431na15_full.csv', index_col=None)
print(song.shape)

(1550400, 17)


In [ ]:
from  sklearn.preprocessing  import StandardScaler 
from sklearn.model_selection import train_test_split

song = song.sample(frac=1).reset_index(drop=True) 
X = song[[str(i) for i in range(y_lim)]]
y = song['class']

#преобразование one-hot и нормализация
tran = StandardScaler()
X = tran.fit_transform(X)
y = pd.get_dummies(y, columns = labels)
print(y.shape)

X, X_test, y, y_test = train_test_split(X, y, test_size=0.20, random_state=42) #тестовая
X, X_val, y, y_val = train_test_split(X, y, test_size=0.10, random_state=42) #валидационная

n_row = (X.shape[0] // x_lim) * x_lim
new_row = n_row // x_lim
X = X[:n_row, :]
X = np.array(X).reshape(new_row, x_lim, y_lim) #для подачи на входд LTSM
y = y.iloc[:n_row, :]
y = np.array(y).reshape(new_row, x_lim, 12)[:, 0, :]
print(X.shape)
print(y.shape)

n_row = (X_val.shape[0] // x_lim) * x_lim
new_row = n_row // x_lim
X_val = X_val[:(X_val.shape[0] // x_lim) * x_lim, :]
X_val = np.array(X_val).reshape(-1, x_lim, y_lim) #для подачи на входд LTSM
y_val = y_val.iloc[:n_row, :]
y_val = np.array(y_val).reshape(new_row, x_lim, 12)[:, 0, :]
print(X_val.shape)
print(y_val.shape)

n_row = (X_test.shape[0] // x_lim) * x_lim
new_row = n_row // x_lim
X_test = X_test[:(X_test.shape[0] // x_lim) * x_lim, :]
X_test = np.array(X_test).reshape(-1, x_lim, y_lim) #для подачи на входд LTSM
y_test = y_test.iloc[:n_row, :]
y_test = np.array(y_test).reshape(new_row, x_lim, 12)[:, 0, :]
print(X_test.shape)
print(y_test.shape)

(1550400, 12)
(2589, 431, 15)
(2589, 12)
(287, 431, 15)
(287, 12)
(719, 431, 15)
(719, 12)


In [ ]:
from keras import backend as K 

# Метрики 
def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [ ]:
from  tqdm  import tqdm 
from  keras.models  import Sequential 
from  keras.layers  import LSTM, Dense, Dropout, Activation 
from  keras.optimizers  import Adam

In [ ]:
model = Sequential()

model.add(LSTM(1000, return_sequences=False, input_shape=(x_lim, y_lim,), activation = 'tanh'))

model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(12, activation = 'softmax'))


model.compile(loss='categorical_crossentropy', metrics=['accuracy', precision, recall, f1], optimizer='adam')

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(os.path.join(os.getcwd, "lstm-genres-{epoch:02d}-{val_f1:.2f}.hdf5"), # Сохраняем модель с лучшим val_f1
                             monitor='val_f1', save_best_only=True, mode='max', period=1)
history = model.fit(X, y, batch_size=32, epochs=10, validation_data=(X_val, y_val), callbacks = [checkpoint])

Epoch 1/1000
81/81 [==============================] - 5s 64ms/step - loss: 2.3997 - accuracy: 0.1599 - precision: 0.0864 - recall: 0.0027 - f1: 0.0052 - val_loss: 2.5253 - val_accuracy: 0.0732 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1: 0.0000e+00
Epoch 2/1000
81/81 [==============================] - 5s 64ms/step - loss: 2.3840 - accuracy: 0.1626 - precision: 0.0741 - recall: 0.0031 - f1: 0.0059 - val_loss: 2.5295 - val_accuracy: 0.0906 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1: 0.0000e+00
Epoch 3/1000
81/81 [==============================] - 5s 64ms/step - loss: 2.3529 - accuracy: 0.1896 - precision: 0.1605 - recall: 0.0058 - f1: 0.0111 - val_loss: 2.5520 - val_accuracy: 0.0836 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1: 0.0000e+00
Epoch 4/1000
81/81 [==============================] - 5s 64ms/step - loss: 2.3113 - accuracy: 0.2074 - precision: 0.3185 - recall: 0.0174 - f1: 0.0324 - val_loss: 2.6067 - val_accuracy: 0.0871 - val_

KeyboardInterrupt: ignored

In [ ]:
y_pred = model.predict_classes(X_test) #To predict labels
precision_test = precision(y_test, y_pred)
recall_test = recall(y_test, y_pred)
f1_test = f1(y_test, y_pred)
print(f"Precision test: {precision_test}")
print(f"Recall test: {recall_test}")
print(f"f1 test: {f1_test}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('ggplot')

def plot_metrix(ax, x1, x2, title):
    ax.plot(range(1, len(x1) + 1), x1, label='train')
    ax.plot(range(1, len(x2) + 1), x2, label='val')
    ax.set_ylabel(title)
    ax.set_xlabel('Epoch')
    ax.legend()
    ax.margins(0)

def plot_history(history):
    fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(16, 9))
    ax1, ax2, ax3, ax4 = axes.ravel()

    plot_metrix(ax1, history.history['precision'], history.history['val_precision'], 'Precision')
    plot_metrix(ax2, history.history['recall'], history.history['val_recall'], 'Recall')
    plot_metrix(ax3, history.history['f1'], history.history['val_f1'], "$F_1$")
    plot_metrix(ax4, history.history['loss'], history.history['val_loss'], 'Loss')
    plot_metrix(ax4, history.history['accuracy'], history.history['val_accuracy'], 'Accuracy')

    plt.show()

plot_history(history)

In [ ]:
model.load_weights(os.path.join(os.getcwd, "lstm-genres-1-1.hdf5"))